# Data Dictionary:

race_id: str
>Identifier for each race. The first race of the season will be 1. The last race will be 819. 

race: str
>The race number of each racing day.

date: time
>Date of the race.

place: str
>The order of horse complete the race track

horse_id: str
>Identifier for each horse

horse_no: str
>Number of the horse for that race

horse: str
>Name of the horse

jockey: str
>Name of the jockey

trainer: str
>Name of the trainer

actual_weight: float
>Weight that added on the horse

declared_horse_weight: float
>Weight of the horse

draw: str
>The stall where the horse started from.

lbw: float
>Distance between the horse and the winner when winner pass the rush line

running_position: array
>Place of the horse pass different check points

win_odds: float
>Ration of beting this horse if the horse win the 1st place

class: str
>The level/grade of the horse. If there is 'Group' in class, it is a international standard race.

going: str
>Condition of race track

track: str
>Field type of the race.Turf or mud (all weather track).

prize: int
>Prize for winner

location: str
>Location of the racecourse

distance_m: int
>Distance for this race

finish_time: time
>Time to finish the race

finish_time_s: float
>Time to finish the race in second

## Please use the follow dictionary for read_csv
```
types_dict = {'race_id': str, 'race': str, 'date':str, 'place':str, 'horse_id':str, 'horse_no':str,
'horse':str, 'jockey':str, 'trainer':str, 'actual_weight':float, 'declared_horse_weight':float,
'draw':str, 'lbw':float, 'running_position': str,'win_odds':float, 'class':str, 'going':str,
'track':str, 'prize':int, 'location':str, 'distance_m':int, 'finish_time':str, 'finish_time_s':float}
parse_dates = ['date', 'finish_time']
df=pd.read_csv('full_season_performance_clean.csv',dtype=types_dict,parse_dates=parse_dates)
```

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
raw = pd.read_csv('data/raw/full_season_performance.csv')
raw.head()

,place,horse_no,horse,jockey,trainer,actual_weight,declared_horse_weight,draw,lbw,running_position,finish_time,win_odds,race,class,going,turf,prize,location,date
0,1,3.0,GOOD RUNNERS WAY(C127),Z Purton,C S Shum,131,1072,6,-,3 ...,1:35.48,2.2,1(1),Class 5 - 1600M - (40-0),GOOD,"TURF - ""B"" Course","HK$ 725,000",Sha Tin,2019/09/01
1,2,4.0,REGENCY GEM(C035),J Moreira,W Y So,129,1179,7,SH,9 ...,1:35.50,4.9,1(1),Class 5 - 1600M - (40-0),GOOD,"TURF - ""B"" Course","HK$ 725,000",Sha Tin,2019/09/01
2,3,7.0,PROUD SKY(V406),K Teetan,L Ho,121,1053,4,1/2,5 ...,1:35.57,18,1(1),Class 5 - 1600M - (40-0),GOOD,"TURF - ""B"" Course","HK$ 725,000",Sha Tin,2019/09/01
3,4,2.0,LE PANACHE(V369),A Sanna,D J Whyte,132,1076,3,1-1/4,4 ...,1:35.68,5.7,1(1),Class 5 - 1600M - (40-0),GOOD,"TURF - ""B"" Course","HK$ 725,000",Sha Tin,2019/09/01
4,5,1.0,GOLD VELVET(V400),L Hewitson,A T Millard,133,1155,8,1-1/2,8 ...,1:35.71,7,1(1),Class 5 - 1600M - (40-0),GOOD,"TURF - ""B"" Course","HK$ 725,000",Sha Tin,2019/09/01


In [3]:
raw.shape

(11459, 19)

In [4]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11459 entries, 0 to 11458
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   place                  11459 non-null  object 
 1   horse_no               11333 non-null  float64
 2   horse                  11459 non-null  object 
 3   jockey                 11459 non-null  object 
 4   trainer                11459 non-null  object 
 5   actual_weight          11459 non-null  int64  
 6   declared_horse_weight  11459 non-null  object 
 7   draw                   11459 non-null  object 
 8   lbw                    11459 non-null  object 
 9   running_position       11459 non-null  object 
 10  finish_time            11459 non-null  object 
 11  win_odds               11459 non-null  object 
 12  race                   11459 non-null  object 
 13  class                  11459 non-null  object 
 14  going                  11459 non-null  object 
 15  tu

In [5]:
#Drop samples with place has no digits
#Change places with DH to digit only
#reset index of dataframe
df_place=raw.copy()
df_place=df_place.drop_duplicates()
df_place=df_place[(df_place['place']!='WV')&(df_place['place']!='WV-A')]
df_place=df_place[(df_place['place']!='WX')&(df_place['place']!='WX-A')&(df_place['place']!='WXNR')]
df_place=df_place[(df_place['place']!='PU')&(df_place['place']!='UR')&(df_place['place']!='DNF')&(df_place['place']!='TNP')]
df_place['place']=df_place['place'].str.extract(r'(\d+)')
df_place=df_place.reset_index(drop=True)
#Change horse no. to int
df_place['horse_no']=df_place['horse_no'].astype(int).astype(str)

In [6]:
#split horse name into house name and horse id
df_horse = df_place.copy()
df_horse['horse_id']=df_horse['horse'].str.extract(r'\((.+)\)')
df_horse['horse']=df_horse['horse'].str.extract(r'^(.+)\(')

#change actual weight into float
df_horse['actual_weight']=df_horse['actual_weight'].astype(float)

#change declared horse weight into float
df_horse['declared_horse_weight']=df_horse['declared_horse_weight'].astype(float)

In [7]:
#change lbw into float
df_lbw=df_horse.copy()
df_lbw.loc[df_lbw['lbw']=='-','lbw']='0'

def lbw_to_float(lbw_string):
    if lbw_string=='N':
        return 0.33
    elif lbw_string=='SH':
        return 0.1
    elif lbw_string=='HD':
        return 0.25
    elif lbw_string=='NOSE':
        return 0.02
    elif lbw_string=='ML':
        return 150
    nums = [int(i) for i in re.findall('\d+',lbw_string)]
    if len(nums)==1:
        return nums[0]
    elif len(nums)==2:
        return nums[0]/nums[1]
    elif len(nums)==3:
        return nums[0]+nums[1]/nums[2]
    else:
        return 999999

df_lbw['lbw']=df_lbw['lbw'].apply(lbw_to_float)

#change running position into array
df_lbw['running_position']=df_lbw['running_position'].str.findall(r'(\d+)')

In [8]:
#change finish_time into second with type float
df_time=df_lbw.copy()
def get_sec(finish_time):
    result=[float(i) for i in finish_time.split(':')]
    return result[0]*60+result[1]
df_time['finish_time_s']=df_time['finish_time'].apply(get_sec)
df_time['finish_time']=pd.to_datetime(df_time['finish_time'],format='%M:%S.%f')
df_time['win_odds']=df_time['win_odds'].astype(float)

In [9]:
#split race and race_id
df_race=df_time.copy()
df_race['race_id']=df_race['race'].str.extract(r'\((\d+)')
df_race['race']=df_race['race'].str.extract(r'(\d+)\(')

In [10]:
#extract number and class from original class data
def get_level(race_class):
    result=race_class.split('-')
    return result[0].strip()

def get_distance(race_class):
    result=race_class.split('-')
    return int(result[1].strip()[:-1])

df_race['distance_m']=df_race['class'].apply(get_distance)
df_race['class']=df_race['class'].apply(get_level)

In [11]:
#extract digit from prize
#transform date to datetime type
df_prize=df_race.copy()
df_prize['prize']=df_prize['prize'].str.replace(r'[^0-9]', "").astype(int)
df_prize['date']=pd.to_datetime(df_prize['date'])

In [12]:
#rename column turf to track 
#reorder the columns
df_final=df_prize.copy()
df_final['track']=df_final['turf']
df_final=df_final.drop(columns='turf',axis=1)
df_final=df_final[['race_id', 'race', 'date', 'place', 'horse_id', 'horse_no', 'horse', 'jockey', 'trainer', 'actual_weight',
       'declared_horse_weight', 'draw', 'lbw', 'running_position', 'win_odds', 'class', 'going', 'track',
       'prize', 'location', 'distance_m', 'finish_time', 'finish_time_s']]

In [13]:
df_final

,race_id,race,date,place,horse_id,horse_no,horse,jockey,trainer,actual_weight,...,running_position,win_odds,class,going,track,prize,location,distance_m,finish_time,finish_time_s
0,1,1,2019-09-01,1,C127,3,GOOD RUNNERS WAY,Z Purton,C S Shum,131.0,...,"[3, 3, 2, 1]",2.2,Class 5,GOOD,"TURF - ""B"" Course",725000,Sha Tin,1600,1900-01-01 00:01:35.480,95.48
1,1,1,2019-09-01,2,C035,4,REGENCY GEM,J Moreira,W Y So,129.0,...,"[9, 8, 8, 2]",4.9,Class 5,GOOD,"TURF - ""B"" Course",725000,Sha Tin,1600,1900-01-01 00:01:35.500,95.50
2,1,1,2019-09-01,3,V406,7,PROUD SKY,K Teetan,L Ho,121.0,...,"[5, 4, 3, 3]",18.0,Class 5,GOOD,"TURF - ""B"" Course",725000,Sha Tin,1600,1900-01-01 00:01:35.570,95.57
3,1,1,2019-09-01,4,V369,2,LE PANACHE,A Sanna,D J Whyte,132.0,...,"[4, 6, 6, 4]",5.7,Class 5,GOOD,"TURF - ""B"" Course",725000,Sha Tin,1600,1900-01-01 00:01:35.680,95.68
4,1,1,2019-09-01,5,V400,1,GOLD VELVET,L Hewitson,A T Millard,133.0,...,"[8, 7, 7, 5]",7.0,Class 5,GOOD,"TURF - ""B"" Course",725000,Sha Tin,1600,1900-01-01 00:01:35.710,95.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10202,828,9,2020-07-15,8,A177,6,ENCORE BOY,T H So,P F Yiu,120.0,...,"[5, 3, 8]",11.0,Class 2,GOOD TO FIRM,"TURF - ""B"" Course",2100000,Happy Valley,1200,1900-01-01 00:01:09.100,69.10
10203,828,9,2020-07-15,9,B285,12,RACING FIGHTER,M F Poon,C H Yip,113.0,...,"[12, 12, 9]",20.0,Class 2,GOOD TO FIRM,"TURF - ""B"" Course",2100000,Happy Valley,1200,1900-01-01 00:01:09.240,69.24
10204,828,9,2020-07-15,10,A379,9,SUNNY BOY,V Borges,J Moore,117.0,...,"[4, 4, 10]",10.0,Class 2,GOOD TO FIRM,"TURF - ""B"" Course",2100000,Happy Valley,1200,1900-01-01 00:01:09.360,69.36
10205,828,9,2020-07-15,11,A293,8,SAUL'S SPECIAL,K H Chan,C W Chang,110.0,...,"[11, 11, 11]",47.0,Class 2,GOOD TO FIRM,"TURF - ""B"" Course",2100000,Happy Valley,1200,1900-01-01 00:01:09.930,69.93


In [14]:
df_final.to_csv('data/performance.csv',index=False)

In [15]:
df_final.info(),df_final.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10207 entries, 0 to 10206
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   race_id                10207 non-null  object        
 1   race                   10207 non-null  object        
 2   date                   10207 non-null  datetime64[ns]
 3   place                  10207 non-null  object        
 4   horse_id               10207 non-null  object        
 5   horse_no               10207 non-null  object        
 6   horse                  10207 non-null  object        
 7   jockey                 10207 non-null  object        
 8   trainer                10207 non-null  object        
 9   actual_weight          10207 non-null  float64       
 10  declared_horse_weight  10207 non-null  float64       
 11  draw                   10207 non-null  object        
 12  lbw                    10207 non-null  float64       
 13  r

(None, (10207, 23))

In [16]:
types_dict = {'race_id': str, 'race': str, 'date':str, 'place':str, 'horse_id':str, 'horse_no':str,
              'horse':str, 'jockey':str, 'trainer':str, 'actual_weight':float, 'declared_horse_weight':float,
              'draw':str, 'lbw':float, 'running_position': str,'win_odds':float, 'class':str, 'going':str,
              'track':str, 'prize':int, 'location':str, 'distance_m':int, 'finish_time':str, 'finish_time_s':float}
parse_dates = ['date', 'finish_time']

df=pd.read_csv('data/performance.csv',dtype=types_dict,parse_dates=parse_dates)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10207 entries, 0 to 10206
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   race_id                10207 non-null  object        
 1   race                   10207 non-null  object        
 2   date                   10207 non-null  datetime64[ns]
 3   place                  10207 non-null  object        
 4   horse_id               10207 non-null  object        
 5   horse_no               10207 non-null  object        
 6   horse                  10207 non-null  object        
 7   jockey                 10207 non-null  object        
 8   trainer                10207 non-null  object        
 9   actual_weight          10207 non-null  float64       
 10  declared_horse_weight  10207 non-null  float64       
 11  draw                   10207 non-null  object        
 12  lbw                    10207 non-null  float64       
 13  r

In [18]:
df

,race_id,race,date,place,horse_id,horse_no,horse,jockey,trainer,actual_weight,...,running_position,win_odds,class,going,track,prize,location,distance_m,finish_time,finish_time_s
0,1,1,2019-09-01,1,C127,3,GOOD RUNNERS WAY,Z Purton,C S Shum,131.0,...,"['3', '3', '2', '1']",2.2,Class 5,GOOD,"TURF - ""B"" Course",725000,Sha Tin,1600,1900-01-01 00:01:35.480,95.48
1,1,1,2019-09-01,2,C035,4,REGENCY GEM,J Moreira,W Y So,129.0,...,"['9', '8', '8', '2']",4.9,Class 5,GOOD,"TURF - ""B"" Course",725000,Sha Tin,1600,1900-01-01 00:01:35.500,95.50
2,1,1,2019-09-01,3,V406,7,PROUD SKY,K Teetan,L Ho,121.0,...,"['5', '4', '3', '3']",18.0,Class 5,GOOD,"TURF - ""B"" Course",725000,Sha Tin,1600,1900-01-01 00:01:35.570,95.57
3,1,1,2019-09-01,4,V369,2,LE PANACHE,A Sanna,D J Whyte,132.0,...,"['4', '6', '6', '4']",5.7,Class 5,GOOD,"TURF - ""B"" Course",725000,Sha Tin,1600,1900-01-01 00:01:35.680,95.68
4,1,1,2019-09-01,5,V400,1,GOLD VELVET,L Hewitson,A T Millard,133.0,...,"['8', '7', '7', '5']",7.0,Class 5,GOOD,"TURF - ""B"" Course",725000,Sha Tin,1600,1900-01-01 00:01:35.710,95.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10202,828,9,2020-07-15,8,A177,6,ENCORE BOY,T H So,P F Yiu,120.0,...,"['5', '3', '8']",11.0,Class 2,GOOD TO FIRM,"TURF - ""B"" Course",2100000,Happy Valley,1200,1900-01-01 00:01:09.100,69.10
10203,828,9,2020-07-15,9,B285,12,RACING FIGHTER,M F Poon,C H Yip,113.0,...,"['12', '12', '9']",20.0,Class 2,GOOD TO FIRM,"TURF - ""B"" Course",2100000,Happy Valley,1200,1900-01-01 00:01:09.240,69.24
10204,828,9,2020-07-15,10,A379,9,SUNNY BOY,V Borges,J Moore,117.0,...,"['4', '4', '10']",10.0,Class 2,GOOD TO FIRM,"TURF - ""B"" Course",2100000,Happy Valley,1200,1900-01-01 00:01:09.360,69.36
10205,828,9,2020-07-15,11,A293,8,SAUL'S SPECIAL,K H Chan,C W Chang,110.0,...,"['11', '11', '11']",47.0,Class 2,GOOD TO FIRM,"TURF - ""B"" Course",2100000,Happy Valley,1200,1900-01-01 00:01:09.930,69.93
